In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.5/586.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 27.9 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from PIL import Image

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.0.105 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


In [ ]:
ipaths=[]
tpaths=[]
for dirname, _, filenames in os.walk('/content/drive/MyDrive/PFE Master/data/Fire'):
    for filename in filenames:
        if filename[-4:]=='.png':
            ipaths+=[os.path.join(dirname, filename)]
        elif filename[-4:]=='.txt':
            tpaths+=[os.path.join(dirname, filename)]
ipaths=sorted(ipaths)
tpaths=sorted(tpaths)
print(ipaths[0])
print(len(ipaths))

/content/drive/MyDrive/PFE Master/data/Fire/FireV10_0.png
10040


In [ ]:
print(len(tpaths))
print(tpaths[1039])
print(len(filenames))
print(filenames[20079])

10040
/content/drive/MyDrive/PFE Master/data/Fire/FireV12_311.txt
20080
FireV2_112.png


In [ ]:
boxdata=[]
boxfile=[]
for i in range(len(tpaths)):
    file=tpaths[i]
    boxdata+=[np.loadtxt(file)]
    boxfile+=[file[0:-4].split('/')[-1]]
print(boxdata[0])

[          0     0.53906     0.31806      0.0125    0.052778]


In [ ]:
BOX=pd.DataFrame()

for i in range(len(boxdata)):
    if type(boxdata[i][0])==np.float64:
        add=pd.DataFrame([boxdata[i]])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])
    else:
        add=pd.DataFrame(boxdata[i])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])

BOX2=BOX.reset_index(drop=True)
display(BOX2)

,0,1,2,3,4,5
0,0.0,0.539062,0.318056,0.012500,0.052778,FireV10_0
1,0.0,0.537500,0.318056,0.015625,0.052778,FireV10_1
2,0.0,0.539062,0.319444,0.015625,0.055556,FireV10_10
3,0.0,0.533594,0.326389,0.020313,0.075000,FireV10_100
4,0.0,0.533594,0.323611,0.020313,0.080556,FireV10_101
...,...,...,...,...,...,...
10035,0.0,0.576562,0.269444,0.015625,0.122222,FireV9_95
10036,0.0,0.576562,0.270833,0.015625,0.119444,FireV9_96
10037,0.0,0.578125,0.270833,0.015625,0.119444,FireV9_97
10038,0.0,0.576562,0.273611,0.015625,0.119444,FireV9_98


In [ ]:
class_map = {0:'Fire'}

In [ ]:
for i in range(len(BOX2)):
    BOX2.iloc[i,0]=class_map[int(BOX2.iloc[i,0])]
display(BOX2)
display(BOX2.iloc[:,0].value_counts())

,0,1,2,3,4,5
0,Fire,0.539062,0.318056,0.012500,0.052778,FireV10_0
1,Fire,0.537500,0.318056,0.015625,0.052778,FireV10_1
2,Fire,0.539062,0.319444,0.015625,0.055556,FireV10_10
3,Fire,0.533594,0.326389,0.020313,0.075000,FireV10_100
4,Fire,0.533594,0.323611,0.020313,0.080556,FireV10_101
...,...,...,...,...,...,...
10035,Fire,0.576562,0.269444,0.015625,0.122222,FireV9_95
10036,Fire,0.576562,0.270833,0.015625,0.119444,FireV9_96
10037,Fire,0.578125,0.270833,0.015625,0.119444,FireV9_97
10038,Fire,0.576562,0.273611,0.015625,0.119444,FireV9_98


Fire    10040
Name: 0, dtype: int64

In [ ]:
def draw_box(n0):

    ipath=ipaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath[0:-4].split('/')[-1]

    if BOX2[BOX2[5]==file] is not None:
        box=BOX2[BOX2[5]==file]
        box=box.reset_index(drop=True)

        for i in range(len(box)):
            label=box.iloc[i,0]
            x=box.iloc[i,1]
            y=box.iloc[i,2]
            w=box.iloc[i,3]
            h=box.iloc[i,4]
            x0=((x-w/2)*W).astype(int)
            y0=((y-h/2)*H).astype(int)
            x1=((x+w/2)*W).astype(int)
            y1=((y+h/2)*H).astype(int)
            if label=='Fire':
                cv2.rectangle(image,(x0,y0),(x1,y1),(0,255,0),5) #green

    #plt.imshow(image)
    #plt.show()

    return image

In [ ]:
images1=[]
for i in tqdm(range(len(ipaths))):
    images1+=[draw_box(i)]

100%|██████████| 10040/10040 [02:18<00:00, 72.47it/s]


In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
def create_animation(ims):
    fig=plt.figure(figsize=(10,6))
    #plt.axis('off')
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    plt.close()
    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        return [im]
    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000//2)

In [ ]:
create_animation(images1)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import os
import random
import shutil

# Set the path to the directory containing your data
data_dir = "/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets"

# Set the path to the directory where you want to save the train, validation, and test sets
train_dir = "/content/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/train"
valid_dir = "/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/validation"
test_dir = "/content/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/test"

# Set the split ratios (e.g., 0.6 for train, 0.2 for validation, and 0.2 for test)
train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15

# Create train, validation, and test directories if they don't exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(valid_dir):
    os.makedirs(valid_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Iterate through each file in the data directory
for file_name in os.listdir(data_dir):
    if file_name.endswith(".png"):
        # Check if the corresponding label file exists
        label_file = file_name.replace(".png", ".txt")
        if not os.path.exists(os.path.join(data_dir, label_file)):
            print(f"Label file not found for {file_name}. Skipping...")
            continue

        # Decide whether to put the file in the train, validation, or test set
        rand_num = random.uniform(0, 1)
        if rand_num < train_ratio:
            dest_dir = train_dir
        elif rand_num < train_ratio + valid_ratio:
            dest_dir = valid_dir
        else:
            dest_dir = test_dir

        # Move the image file to the train, validation, or test directory
        shutil.move(os.path.join(data_dir, file_name), os.path.join(dest_dir, file_name))

        # Move the label file to the train, validation, or test directory
        shutil.move(os.path.join(data_dir, label_file), os.path.join(dest_dir, label_file))


In [ ]:
import yaml

fire_yaml = dict(
    train ='/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/train',
    val ='/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/validation',
    test='/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/test',
    nc =1,
    names =['Fire']
)

with open('fire.yaml', 'w') as outfile:
    yaml.dump(fire_yaml, outfile, default_flow_style=True)

%cat fire.yaml

{names: [Fire], nc: 1, test: /content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/test,
  train: /content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/train,
  val: /content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/validation}


In [ ]:
import os

# Set the path to the directory
directory = "/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/test"

# Get the list of files in the directory
file_list = os.listdir(directory)

# Delete files with the ".txt" extension
for file_name in file_list:
    if file_name.endswith(".txt"):
        file_path = os.path.join(directory, file_name)
        os.remove(file_path)


In [ ]:
path="/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/datasets/train"
file_list = os.listdir(path)
print(len(file_list))

In [ ]:
model = YOLO("yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 107MB/s]


In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=fire.yaml epochs=20

Ultralytics YOLOv8.0.105 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=fire.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=

In [ ]:
import torch

In [ ]:
# Assuming you have the model weights stored in a variable called 'model_weights'
torch.save(model, '/content/drive/MyDrive/PFE Master/Project/YOLOV8FireDetection_V1/FireDetection.pt')

In [ ]:
!zip -r /content/runs.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/predict/ (stored 0%)
  adding: content/runs/detect/predict/FireV5_232.png (deflated 6%)
  adding: content/runs/detect/predict/FireV24_192.png (deflated 6%)
  adding: content/runs/detect/predict/FireV19_264.png (deflated 7%)
  adding: content/runs/detect/predict/FireV19_272.png (deflated 6%)
  adding: content/runs/detect/predict/FireV7_395.png (deflated 5%)
  adding: content/runs/detect/predict/FireV6_170.png (deflated 4%)
  adding: content/runs/detect/predict/FireV27_61.png (deflated 3%)
  adding: content/runs/detect/predict/FireV10_315.png (deflated 6%)
  adding: content/runs/detect/predict/FireV13_240.png (deflated 5%)
  adding: content/runs/detect/predict/FireV25_374.png (deflated 5%)
  adding: content/runs/detect/predict/FireV3_85.png (deflated 7%)
  adding: content/runs/detect/predict/FireV18_22.png (deflated 4%)
  adding: content/runs/detect/predict/FireV17_180.png (deflat

In [ ]:
from google.colab import files

files.download('/content/runs.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tpaths2=[]
for dirname, _, filenames in os.walk('/content/runs/detect/train'):
    for filename in filenames:
        if filename[-4:]=='.png' or filename[-4:]=='.jpg':
            tpaths2+=[(os.path.join(dirname, filename))]
tpaths2=sorted(tpaths2)
print(tpaths2[0])

/content/runs/detect/train/F1_curve.png


In [ ]:
for path in tpaths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

Predict

In [ ]:
best_path0="/content/runs/detect/train/weights/best.pt"
source0="/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test"

In [ ]:
ppaths=[]
for dirname, _, filenames in os.walk(source0):
    for filename in filenames:
        if filename[-4:]=='.png':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths)

['/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_108.png', '/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_121.png', '/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_135.png', '/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_136.png', '/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_141.png', '/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_148.png', '/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_149.png', '/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_150.png', '/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_161.png', '/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_165.png', '/content/drive/MyD

In [ ]:
model_2 = YOLO(best_path0)

In [ ]:
!yolo task=detect mode=predict model={best_path0} conf=0.0062 source={source0}

Ultralytics YOLOv8.0.105 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients

image 1/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_108.png: 384x640 1 Fire, 57.6ms
image 2/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_121.png: 384x640 1 Fire, 14.4ms
image 3/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_135.png: 384x640 1 Fire, 5.6ms
image 4/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_136.png: 384x640 1 Fire, 7.3ms
image 5/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_141.png: 384x640 1 Fire, 5.6ms
image 6/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_148.png: 384x640 1 Fire, 5.6ms
image 7/1496 /content/drive/MyDrive/PFE_Master/Project/Y

Result of predection

In [ ]:
results = model2.predict(source0,conf=0.0062)
print(len(results))



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_108.png: 384x640 1 Fire, 9.7ms
image 2/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_121.png: 384x640 1 Fire, 5.7ms
image 3/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/test/FireV10_135.png: 384x640 1 Fire, 5.6ms
image 4/1496 /content/drive/MyDrive/PFE_Master/Project/YOLOV8FireD

1496


In [ ]:
import torch
import pandas as pd

PBOX=pd.DataFrame(columns=range(6))
for i in range(len(results)):
    arri = pd.DataFrame(results[i].boxes.boxes.cpu().detach().numpy()).astype(float)
    path=ppaths[i]
    file=path.split('/')[-1]
    arri=arri.assign(file=file)
    arri=arri.assign(i=i)
    PBOX=pd.concat([PBOX,arri],axis=0)
PBOX.columns=['x','y','x2','y2','confidence','class','file','i',]
display(PBOX)

WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.
WARNING ⚠️

,x,y,x2,y2,confidence,class,file,i
0,333.330566,96.376602,348.005981,131.441055,0.849645,0.0,FireV10_108.png,0.0
0,333.153900,98.247528,347.320770,133.693802,0.816342,0.0,FireV10_121.png,1.0
0,332.147858,105.012596,346.248016,135.349091,0.842280,0.0,FireV10_135.png,2.0
0,332.158691,105.725487,346.044556,135.547821,0.845605,0.0,FireV10_136.png,3.0
0,331.765015,106.528732,345.768066,136.157700,0.843174,0.0,FireV10_141.png,4.0
...,...,...,...,...,...,...,...,...
0,364.668884,72.447586,375.111145,118.893112,0.800239,0.0,FireV9_88.png,1494.0
1,53.222652,100.808456,89.832977,167.060440,0.013102,0.0,FireV9_88.png,1494.0
2,38.174911,96.845810,86.259064,168.068771,0.009468,0.0,FireV9_88.png,1494.0
3,38.607254,109.060638,85.619324,180.119507,0.006912,0.0,FireV9_88.png,1494.0


In [ ]:
PBOX['class']=PBOX['class'].apply(lambda x: class_map[int(x)])
PBOX=PBOX.reset_index(drop=True)
display(PBOX)
display(PBOX['class'].value_counts())

,x,y,x2,y2,confidence,class,file,i
0,333.330566,96.376602,348.005981,131.441055,0.849645,Fire,FireV10_108.png,0.0
1,333.153900,98.247528,347.320770,133.693802,0.816342,Fire,FireV10_121.png,1.0
2,332.147858,105.012596,346.248016,135.349091,0.842280,Fire,FireV10_135.png,2.0
3,332.158691,105.725487,346.044556,135.547821,0.845605,Fire,FireV10_136.png,3.0
4,331.765015,106.528732,345.768066,136.157700,0.843174,Fire,FireV10_141.png,4.0
...,...,...,...,...,...,...,...,...
2007,364.668884,72.447586,375.111145,118.893112,0.800239,Fire,FireV9_88.png,1494.0
2008,53.222652,100.808456,89.832977,167.060440,0.013102,Fire,FireV9_88.png,1494.0
2009,38.174911,96.845810,86.259064,168.068771,0.009468,Fire,FireV9_88.png,1494.0
2010,38.607254,109.060638,85.619324,180.119507,0.006912,Fire,FireV9_88.png,1494.0


Fire    2012
Name: class, dtype: int64

In [ ]:
def draw_box2(n0):

    ipath=ppaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]

    if PBOX[PBOX['file']==file] is not None:
        box=PBOX[PBOX['file']==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.iloc[i,5]
            x=int(box.iloc[i,0])
            y=int(box.iloc[i,1])
            x2=int(box.iloc[i,2])
            y2=int(box.iloc[i,3])
            #print(label,x,y,x2,y2)

            if label=='Fire':
                cv2.rectangle(image,(x,y),(x2,y2),(0,255,0),5) #green

    #plt.imshow(image)
    #plt.show()

    return image

In [ ]:
images2=[]
for i in tqdm(range(len(ppaths))):
    images2+=[draw_box2(i)]

100%|██████████| 1496/1496 [00:15<00:00, 96.78it/s] 


In [ ]:
create_animation(images2)


Output hidden; open in https://colab.research.google.com to view.